### Reading through Python Cookbook, notedown some interesting receipe

In [15]:
# 1.8
# intersting to know dictionary can flip over key-value with zip
# afterward can also apply min/max/sort function to the result, pretty handy
prices = {
'ACME': 45.23,
'AAPL': 612.78,
'IBM': 205.55,
'HPQ': 37.20,
'FB': 10.75
} 

for val in zip(prices.values(), prices.keys()):
    print (val)

prices_sorted = sorted(zip(prices.values(), prices.keys()))
print(prices_sorted)



(45.23, 'ACME')
(612.78, 'AAPL')
(205.55, 'IBM')
(37.2, 'HPQ')
(10.75, 'FB')
[(10.75, 'FB'), (37.2, 'HPQ'), (45.23, 'ACME'), (205.55, 'IBM'), (612.78, 'AAPL')]


In [32]:
# 1.9 Dictionary Keys and Values is actually an object and can use some basic set-like ops
a = {
'x' : 1,
'y' : 2,
'z' : 3
}
b = {
'w' : 10,
'x' : 11,
'y' : 2
}

# Find keys in common
print(a.keys() & b.keys())
 # { 'x', 'y' }
# Find keys in a that are not in b
print(a.keys() - b.keys())
 # { 'z' }
# Find (key,value) pairs in common
print(a.items() & b.items()) # { ('y', 2) }


{'x', 'y'}
{'z'}
{('y', 2)}


In [30]:
# 1.20 ChainMap merge two dict without actually merging it (efficient), any change will
# be place on the first dict
from collections import ChainMap
values = ChainMap()
values['x'] = 1
values = values.new_child()
values['x'] = 2
values

ChainMap({'x': 2}, {'x': 1})

In [ ]:
values = ChainMap()
>>> values = ChainMap()
>>> values['x'] = 1
>>> # Add a new mapping
>>> values = values.new_child()
>>> values['x'] = 2
>>> # Add a new mapping
>>> values = values.new_child()
>>> values['x'] = 3
>>> values
